In [7]:
import pandas as pd 
docs = pd.read_csv("data/index_docs.csv")

In [3]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

import yake
kw_extractor = yake.KeywordExtractor(top=2, lan='en')
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

In [5]:
import tqdm.notebook as tqdm
generated_queries = []

for i,r in tqdm.tqdm(docs.iterrows(), total=len(docs)):
    header = r.title
    doc_uid = r.uuid 
    
    # generate paraphrases 
    phrases = parrot.augment(input_phrase=header, use_gpu=False, do_diverse=False)    
    for phrase in phrases[:3]:
        generated_queries.append({"query": phrase[0], "type": "paraphrase", "gold_uid": doc_uid})
    
queries_df = pd.DataFrame(generated_queries)
queries_df.to_csv("queries.csv")

  0%|          | 0/847 [00:00<?, ?it/s]

In [37]:
import pandas as pd 
queries_df = pd.read_csv("data/queries.csv")

queries_df = queries_df.query("type != 'keyword'")

In [38]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [46]:
from spacy.lang.en.stop_words import STOP_WORDS
import string

f = lambda x: ''.join([i for i in x if i not in string.punctuation])

generated_kw = []
for i,r in queries_df.iterrows():
    
    q =r.query
    
    new_string = []
    for tok in nlp(f(q.lower())):
        if tok.text not in STOP_WORDS:
            new_string.append(tok.text)
    new_string = " ".join(new_string)
    
    generated_kw.append({"query": new_string, "type":  "keyword", "gold_uid": r.gold_uid})

kw_df = pd.DataFrame(generated_kw)
kw_df

,query,type,gold_uid
0,tell best way find local bank regulator,keyword,hj3Rfuw4Ma6B9MN5xKJY8J
1,tell correct way find state bank regulator,keyword,hj3Rfuw4Ma6B9MN5xKJY8J
2,find local bank regulator,keyword,hj3Rfuw4Ma6B9MN5xKJY8J
3,way contact debt collector payments deceased r...,keyword,KitTQDCXSTZNxUhhMwPHpM
4,contact debt collector deceased relatives debts,keyword,KitTQDCXSTZNxUhhMwPHpM
...,...,...,...
2366,children learn money,keyword,meHjq8TYuATLKfVBkF9Snd
2367,tell best way help children learn money works,keyword,nn32fn7JoQ96DK9kGXYBJx
2368,help children learn lot money 6 8 years,keyword,nn32fn7JoQ96DK9kGXYBJx
2369,children know money age 6,keyword,nn32fn7JoQ96DK9kGXYBJx


In [47]:
final = pd.concat([queries_df, kw_df])

In [49]:
final = final[["query", "type", "gold_uid"]]

In [51]:
final.sort_values("gold_uid")

,query,type,gold_uid
2486,can i use the military savings deposit program...,paraphrase,25vDNygHDHdWsFqxW56Cj5
2485,can you use the military savings deposit progr...,paraphrase,25vDNygHDHdWsFqxW56Cj5
1450,use military savings deposit program sdp deplo...,keyword,25vDNygHDHdWsFqxW56Cj5
1451,use military savings deposit program m deploye...,keyword,25vDNygHDHdWsFqxW56Cj5
2003,received bill month payment day usual bill cam...,keyword,2D5XoeFFiXNoafhQ6TG8jH
...,...,...,...
394,tell me the best thing to do after a disaster?,paraphrase,oQRVVCuoXw5tnSo4ZtJshb
234,tell best way protect money disaster strikes,keyword,oQRVVCuoXw5tnSo4ZtJshb
395,how should i prepare for a disaster?,paraphrase,oQRVVCuoXw5tnSo4ZtJshb
2494,when i made a payment online through my bank o...,paraphrase,oUfQSTwbJSDEFy6UpX2qJZ


In [59]:
final.sample(800).to_csv("data/query_sample.csv", index=False)